In [1]:
import os
from glob import glob

# unpack the dataset from https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz and store the folder 'aclImdb' in the same folder as this script

def read_data(dataset):
    texts = []
    labels = []
    for label in ['pos', 'neg']:
        for file in glob(os.path.join('aclImdb',dataset,label,'*.txt')):
            with open(file) as f:
                texts.append(f.read())
                labels.append(label)
    return texts, labels

X_train_fulltext, y_train = read_data('train')
X_test_fulltext, y_test= read_data('test')

In [2]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, classification_report

vectorizer = CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(X_train_fulltext)
X_test = vectorizer.transform(X_test_fulltext)

nb = MultinomialNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[10995  1505]
 [ 3003  9497]]
              precision    recall  f1-score   support

         neg       0.79      0.88      0.83     12500
         pos       0.86      0.76      0.81     12500

   micro avg       0.82      0.82      0.82     25000
   macro avg       0.82      0.82      0.82     25000
weighted avg       0.82      0.82      0.82     25000



In [3]:
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

def short_classification_report (y_test, y_pred):
    print("                \t precision \t recall")
    print("positive reviews:\t {:0.2f} \t\t {:0.2f}".format(metrics.precision_score(y_test,y_pred, pos_label='pos'),
                                                             metrics.recall_score(y_test,y_pred, pos_label='pos')))
    print("negative reviews:\t {:0.2f} \t\t {:0.2f}".format(metrics.precision_score(y_test,y_pred, pos_label='neg'),
                                                            metrics.recall_score(y_test,y_pred, pos_label='neg')))

configurations = [('NB with Count', CountVectorizer(min_df=5, max_df=.5), MultinomialNB()),
                 ('NB with TfIdf', TfidfVectorizer(min_df=5, max_df=.5), MultinomialNB()),
                 ('LogReg with Count', CountVectorizer(min_df=5, max_df=.5), LogisticRegression(solver='liblinear')),
                 ('LogReg with TfIdf', TfidfVectorizer(min_df=5, max_df=.5), LogisticRegression(solver='liblinear'))]

for description, vectorizer, classifier in configurations:
    print(description)
    X_train = vectorizer.fit_transform(X_train_fulltext)
    X_test = vectorizer.transform(X_test_fulltext)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    short_classification_report(y_test, y_pred)
    print('\n')

NB with Count
                	 precision 	 recall
positive reviews:	 0.87 		 0.77
negative reviews:	 0.79 		 0.88


NB with TfIdf
                	 precision 	 recall
positive reviews:	 0.87 		 0.78
negative reviews:	 0.80 		 0.88


LogReg with Count
                	 precision 	 recall
positive reviews:	 0.87 		 0.85
negative reviews:	 0.85 		 0.87


LogReg with TfIdf
                	 precision 	 recall
positive reviews:	 0.89 		 0.88
negative reviews:	 0.88 		 0.89




Same as pipe

In [4]:
from sklearn.pipeline import make_pipeline

for description, vectorizer, classifier in configurations:
    print(description)
    pipe = make_pipeline(vectorizer, classifier)
    pipe.fit(X_train_fulltext, y_train)
    y_pred = pipe.predict(X_test_fulltext)
    short_classification_report(y_test, y_pred)
    print('\n')

NB with Count
                	 precision 	 recall
positive reviews:	 0.87 		 0.77
negative reviews:	 0.79 		 0.88


NB with TfIdf
                	 precision 	 recall
positive reviews:	 0.87 		 0.78
negative reviews:	 0.80 		 0.88


LogReg with Count
                	 precision 	 recall
positive reviews:	 0.87 		 0.85
negative reviews:	 0.85 		 0.87


LogReg with TfIdf
                	 precision 	 recall
positive reviews:	 0.89 		 0.88
negative reviews:	 0.88 		 0.89




## For the sake of comparison: how would an off-the-shelf dictionary approach perform?

In [5]:
from nltk.sentiment import vader

analyzer = vader.SentimentIntensityAnalyzer()
y_vader = []
for review in X_test_fulltext:
    sentiment = analyzer.polarity_scores(review)
    if sentiment['compound']>0:
        y_vader.append('pos')
    elif sentiment['compound']<0:
        y_vader.append('neg')
    else:
        y_vader.append('dont know')
print(confusion_matrix(y_test, y_vader))
print(classification_report(y_test, y_vader))

[[    0     0     0]
 [    6  6668  5826]
 [    5  1748 10747]]
              precision    recall  f1-score   support

   dont know       0.00      0.00      0.00         0
         neg       0.79      0.53      0.64     12500
         pos       0.65      0.86      0.74     12500

   micro avg       0.70      0.70      0.70     25000
   macro avg       0.48      0.46      0.46     25000
weighted avg       0.72      0.70      0.69     25000



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [8]:
import pickle
from sklearn.externals import joblib


# fit vectorizer and model
vectorizer = CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(X_train_fulltext)
nb = MultinomialNB()
nb.fit(X_train, y_train)

# let's save them both
pickle.dump(vectorizer,open("myvectorizer.pkl",mode="wb"))
joblib.dump(nb, "myclassifier.pkl")

#Then, later on, instead of fitting a new vectorizer, you can simply load the old one and use it:

listwithnewdata = ['This is a great movie', 'I hated this one.', 'What an awful fail']

myvectorizer = pickle.load(open("myvectorizer.pkl",mode="rb"))
new_features = vectorizer.transform(listwithnewdata)

myclassifier = joblib.load("myclassifier.pkl")
predictions = myclassifier.predict(new_features)

for review, label in zip(listwithnewdata, predictions):
    print("'{}' probably belongs to class '{}'.".format(review,label))


'This is a great movie' probably belongs to class 'pos'.
'I hated this one.' probably belongs to class 'neg'.
'What an awful fail' probably belongs to class 'neg'.
